In [1]:
import collections
import copy
import cspUtil
import dataUtil
import itertools

In [2]:
def countSequences(inputList, N, gap=0):
    out = collections.defaultdict(float)
    if gap == 0:
        for i in range(len(inputList)-N+1):
            if [inputList[i+k] != None for k in range(N)] == [True for _ in range(N)]:
                seq = tuple([inputList[i+k] for k in range(N)])
                out[seq] += 1
        #print out
    else:
        assert N == 2
        for i in range(len(inputList)-N+1-gap):
            if (inputList[i] != None) and (inputList[i+gap+1] != None):
                seq = tuple([inputList[i], inputList[i+gap+1]])
                out[seq] += 1
        

    factor= 1.0 / sum(out.itervalues())
    for k in out:
          out[k] *= 10 * factor

    return out

In [3]:
def printBinaryFactor(binaryFactor):
    for var1, dict1 in binaryFactor.items():
        for var2, dict2 in dict1.items():
            for val1, dict3 in dict2.items():
                for val2, weight in dict3.items():
                    if weight:
                        print var1, val1, var2, val2, weight
                        
def printUnaryFactor(unaryFactor):
    for var1, dict1 in unaryFactor.items():
        if dict1 is None: continue
        for val1, weight in dict1.items():
            if weight:
                print var1, val1, weight

In [17]:
def createRythmCSP(windowedPrimTrack, window, bi=True, tri=False, bi_gap=True):
    cspRythm = cspUtil.CSP()
    (start, end) = window
    # Creating "units" variables
    transitions = [note[1] for note in windowedPrimTrack]
    for unitNum in range(start, end+1):
        unit = windowedPrimTrack[unitNum]
        cspRythm.add_variable(('U', unitNum), [(unit[0],True),(unit[0],False)])
    
    if bi:
        # Create binary factors proportional to frequence of apparition of pairs
        biSequences = countSequences(transitions, 2)
        for varNum in range(start, end):
            cspRythm.add_binary_factor(('U', varNum), ('U', varNum+1), lambda u1, u2 : biSequences[(u1[1], u2[1])])
        ###### Add corner cases
        cspRythm.add_unary_factor(('U', start), lambda u : biSequences[(windowedPrimTrack[start-1][1], u[1])])
        cspRythm.add_unary_factor(('U', end)  , lambda u : biSequences[(u[1], windowedPrimTrack[end+1][1])])
     
    if bi_gap:  
        # Create binary factors with gap
        for gap in range(1,len(window)):
            biSequencesGap = countSequences(transitions, 2, gap)
            for varNum in range(start, end-gap):
                cspRythm.add_binary_factor(('U', varNum), ('U', varNum+1+gap), lambda u1, u2 : biSequencesGap[(u1[1], u2[1])])
            ###### Add corner cases
            for varNum in range(start, start+gap):
                cspRythm.add_unary_factor(('U', varNum), lambda u : biSequencesGap[(windowedPrimTrack[varNum-1-gap][1], u[1])])
            for varNum in range(end-gap, end): 
                cspRythm.add_unary_factor(('U', varNum)  , lambda u : biSequencesGap[(u[1], windowedPrimTrack[varNum+2+gap][1])])

    if tri:
        ### Compute domain for variables which represent two consecutive pitches ###
        domain2 = tuple(itertools.product((True,False), repeat = 2))

        ### Create variables which represent 2 consecutive pitches as ('B', pitchIdx) ###
        for rythmIdx in range(start, end):
            cspRythm.add_variable(('B', rythmIdx), copy.deepcopy(domain2))
                                                                                                                                                                                                     # Creating "binary" variables for 3-sequences handling
        ### Creating binary factors proportional to frequence of apparition of triads ###                                                                                                                               triSequences = countSequences(windowedPrimTrack, 3)
        freq3 = countSequences(transitions, 3)
        for rythmIdx in range(start, end):
            # Add constraint B_i[0] = U_i
            cspRythm.add_binary_factor(('U', rythmIdx), ('B', rythmIdx), lambda u, b: u[1] == b[0])

            if rythmIdx < end-1:
                # Add consitency contraints B_i[0] = B_{i+1}[1] and frequency factor
                cspRythm.add_binary_factor(('B', rythmIdx), ('B', rythmIdx+1), \
                                        lambda b1, b2: (b1[0] == b2[1]) * freq3[(b1[1], b1[0], b2[0])]) 
                
            
        # Add corner cases
        cspRythm.add_unary_factor(('B', start), lambda b: (b[1] == transitions[start-1]) \
                          * freq3[(transitions[start-2], transitions[start-1], b[0])])
        cspRythm.add_unary_factor(('B', end-1), lambda b: freq3[(b[1], b[0], transitions[end+1])] \
                     * freq3[(b[0], transitions[end+1], transitions[end+2])])

    return cspRythm

In [5]:
def createPitchCSP(windowedPitchPrimTrack, weight_uni, weight_bi, window, bi=True, tri=False, bi_gap=True):
    (start, end) = window
    cspPitch = cspUtil.CSP()
    pitchTrack = [note[0] for note in windowedPitchPrimTrack]
    # Creating "units" variables #
    # Domains
    minPitch, maxPitch = pitchTrack[0], pitchTrack[0]
    for pitch in pitchTrack:
        if type(pitch) == int:
            minPitch, maxPitch = min(minPitch, pitch), max(maxPitch, pitch)
    domains = tuple(list(range(minPitch, maxPitch+1)) + ['silence'])
    
    for pitchNum in range(start, end+1):
        cspPitch.add_variable(('U', pitchNum), copy.deepcopy(domains))

     #frequency of single apparition
    freq1 = countSequences(pitchTrack, 1)
    for pitchIdx in range(start, end):
        cspPitch.add_unary_factor(('U', pitchIdx), lambda p : freq1[(p,)]**weight_uni)
        
    # Create binary factor: must keep the same pitch if previous duration = True
    for varNum in range(start, end): 
        if windowedPitchPrimTrack[varNum][1]:
            cspPitch.add_binary_factor(('U', varNum), ('U', varNum+1), lambda x, y : x == y)
    ###### Add corner cases
    if windowedPitchPrimTrack[start-1][1]:
        cspPitch.add_unary_factor(('U', start), lambda x : x == windowedPitchPrimTrack[start-1][0])
    if windowedPitchPrimTrack[end][1]:
        cspPitch.add_unary_factor(('U', end), lambda x : x == windowedPitchPrimTrack[end+1][0])

    if bi:
        # Create binary factors proportional to frequence of apparition of pairs
        biSequences = countSequences(pitchTrack, 2)
        for varNum in range(start, end):
            cspPitch.add_binary_factor(('U', varNum), ('U', varNum+1), lambda p1, p2 : biSequences[(p1, p2)]**weight_bi)
        ###### Add corner cases
        cspPitch.add_unary_factor(('U', start), lambda p : biSequences[(pitchTrack[start-1], p)]**weight_bi)
        cspPitch.add_unary_factor(('U', end)  , lambda p : biSequences[(p, pitchTrack[end+1])]**weight_bi)
    
    if bi_gap:
        # Create binary factors with gap
        for gap in range(1,len(window)):
            biSequencesGap = countSequences(pitchTrack, 2, gap)
            for varNum in range(start, end-gap):
                cspPitch.add_binary_factor(('U', varNum), ('U', varNum+1+gap), lambda p1, p2 : biSequencesGap[(p1, p2)]**weight_bi)
            ###### Add corner cases
            for varNum in range(start, start+gap):
                cspPitch.add_unary_factor(('U', varNum), lambda p : biSequencesGap[(pitchTrack[varNum-1-gap], p)]**weight_bi)
            for varNum in range(end-gap, end): 
                cspPitch.add_unary_factor(('U', varNum)  , lambda p : biSequencesGap[(p, pitchTrack[varNum+2+gap])]**weight_bi)

    if tri:
        ### Compute domain for variables which represent two consecutive pitches ###
        domain2 = tuple(itertools.product(domains, repeat = 2))

        ### Create variables which represent 2 consecutive pitches as ('B', pitchIdx) ###
        for pitchIdx in range(start, end):
            cspPitch.add_variable(('B', pitchIdx), copy.deepcopy(domain2))
                                                                                                                                                                                                         # Creating "binary" variables for 3-sequences handling
        ### Creating binary factors proportional to frequence of apparition of triads ###                                                                                                                               triSequences = countSequences(windowedPrimTrack, 3)
        freq3 = countSequences(pitchTrack, 3)
        for pitchIdx in range(start, end):
            # Add constraint B_i[0] = U_i
            cspPitch.add_binary_factor(('U', pitchIdx), ('B', pitchIdx), lambda u, b: u == b[0])

            if pitchIdx < end-1:
                # Add consitency contraints B_i[0] = B_{i+1}[1] and frequency factor
                cspPitch.add_binary_factor(('B', pitchIdx), ('B', pitchIdx+1), \
                                        lambda b1, b2: (b1[0] == b2[1]) * freq3[(b1[1], b1[0], b2[0])]**weight_uni)
        # Add corner cases
        cspPitch.add_unary_factor(('B', start), lambda b: (b[1] == pitchTrack[start-1]) \
                              * freq3[(pitchTrack[start-2], pitchTrack[start-1], b[0])]**weight_uni)
        cspPitch.add_unary_factor(('B', end-1), lambda b: freq3[(b[1], b[0], pitchTrack[end+1])]**weight_uni \
                         * freq3[(b[0], pitchTrack[end+1], pitchTrack[end+2])]**weight_uni)
    
    #printBinaryFactor(cspPitch.binaryFactors)
        
    return cspPitch

In [22]:
def reconstruction(mypattern,  window, weight_uni, weight_bi, trackNum=0, bi=True, tri=False, rhythmRec=False, bi_gap=True):
    rythmUnit, windowedPrimTrack = mypattern.getCorrupt(window, trackNum)
    windowedPitchPrimTrack = windowedPrimTrack[:]
    if rhythmRec:
        # Rhythm reconstruction
        CSP_rythm = createRythmCSP(windowedPrimTrack, window, bi, tri, bi_gap)
        search_rythm = cspUtil.BacktrackingSearch()
        search_rythm.solve(CSP_rythm, mcv = False, ac3 = False)  
        
        for i, note in enumerate(windowedPitchPrimTrack[:]):
            if note == (None, None):
                windowedPitchPrimTrack[i] = search_rythm.optimalAssignment[('U', i)]
    else:
        # No Rhythm reconstruction (rhythm based on the original score)
        for i, note in enumerate(windowedPitchPrimTrack[:]):
            if note == (None, None):
                windowedPitchPrimTrack[i] = (None, mypattern.primTracks[trackNum][i][1])

    # Pitch reconstruction
    CSP_pitch = createPitchCSP(windowedPitchPrimTrack, weight_uni, weight_bi, window, bi, tri, bi_gap)
    search_pitch = cspUtil.BacktrackingSearch()
    search_pitch.solve(CSP_pitch, mcv = False, ac3 = False)
    primReconstruction = windowedPitchPrimTrack[:]
    for i, note in enumerate(primReconstruction[:]):
        if note[0] == None:
            if ('U', i) in search_pitch.optimalAssignment.keys():
                primReconstruction[i] = (search_pitch.optimalAssignment[('U', i)], primReconstruction[i][1])
    
    return primReconstruction

In [23]:
def lossFunction(window, primOriginal, primReconstruction):
    (start, end) = window
    accuracy = 0
    for i in range(start, end+1):
        if primOriginal[i][0] == primReconstruction[i][0]:
            accuracy += 1
        else: 
            print 'Position:', i, 'Original pitch:', primOriginal[i][0], 'Reconstructed pitch:', primReconstruction[i][0]
            
        if primOriginal[i][1] == primReconstruction[i][1]:
            accuracy += 1
        else: 
            print 'Position:', i, 'Original rhythm:', primOriginal[i][1], 'Reconstructed rhythm:', primReconstruction[i][1]
            
    loss = 100 * (1 - accuracy/(2.*(end+1-start)))
    #print 'Loss value (percentage):', loss
    return  loss

In [29]:
#midi_file = "/Users/robin/Desktop/CS221_projet/Open_MIDI_file/ode.mid"
midi_file = "/Users/robin/Desktop/CS221_projet/Open_MIDI_file/ode.mid"
mypattern = dataUtil.MusicPattern(midi_file)
mypattern.midiToLisp()
mypattern.lispToPrim()
mypattern.primToLisp()
mypattern.lispToMidi()

In [30]:
trackNum = 0
weight_uni = 0.5
weight_bi = 2
window = (25, 50)
primReconstruction = reconstruction(mypattern,  window, weight_uni, weight_bi, trackNum=0, bi=True, tri=False, rhythmRec=False, bi_gap=False)
primOriginal = mypattern.primTracks[trackNum]
Loss = lossFunction(window, primOriginal, primReconstruction)
Loss

32.692307692307686

In [33]:
import sys
import numpy as np
log = open("log.txt", "w")
sys.stdout = log

path = '/Users/robin/Desktop/CS221_projet/GitHub/cs221/dat/bach-chorales/chorales.lisp'
weight_uni = 1
weight_bi = 1

results = {}

for chorale in range(100):
    trackNum=0
    bi = True
    rhythmRec = True
    mypattern = dataUtil.BachChorale(path, chorale)
    L = len(mypattern.primTracks[0])
    window_list = [(L/4, L/4 +5), (L/4, L/4 +10), (L/2, L/2 +5), (L/2, L/2 +10), (3*L/4, 3*L/4 +5)]
    if len(mypattern.primTracks[0]) > 3*L/4 +10:
        window_list.append((3*L/4, 3*L/4 +10))
    #if len(mypattern.primTracks[0]) > L/2 +20:
    #    window_list.append((L/2, L/2 +20))
    #if len(mypattern.primTracks[0]) > 3*L/4 +20:
    #    window_list.append((3*L/4, 3*L/4 +20))
    for window in window_list:
        for tri in [True, False]:
            for bi_gap in [True, False]:
                try:
                    primReconstruction = reconstruction(mypattern, window, weight_uni, weight_bi, trackNum, bi, tri, rhythmRec, bi_gap)
                    primOriginal = mypattern.primTracks[trackNum]
                    Loss = lossFunction(window, primOriginal, primReconstruction)
                    results[(chorale, L, window, tri, bi_gap)] = Loss
                    print 'chorale numero: ', chorale
                    print 'chorale length: ', L
                    print 'window: ', window
                    print 'tri: ', tri
                    print 'bi_gap: ', bi_gap
                    print 'Loss: ', Loss
                    print '########################'
                except:
                    pass
# Save             
np.save('results_dict.npy', results) 

In [ ]:
# Load
read_dictionary = np.load('results_dict.npy').item()

In [ ]:
read_dictionary

# Finding optimal weights for unary and binary factors

In [ ]:
trackNum = 0
weight_uni = 1
weight_bi = 1
#window = (20, 30)
for i in range(10,30):
    window = (i, i+10)
    primReconstruction = reconstruction(mypattern,  window, weight_uni, weight_bi, trackNum=0, bi=True, tri=False, rhythmRec=False)
    primOriginal = mypattern.primTracks[trackNum]
    Loss = lossFunction(window, primOriginal, primReconstruction)
    print Loss

In [ ]:
Loss_mean = []
for i in range(len(Weight_factors)):
    weight_uni = Weight_factors[i]
    for j in range(len(Weight_factors)):
        weight_bi = Weight_factors[j]
        Losses = [trainExamples[k][1] for k in range(len(trainExamples)) if (trainExamples[k][0][1] == weight_uni) & (trainExamples[k][0][2] == weight_bi)]
        Loss_mean.append([weight_uni, weight_bi, sum(Losses)/len(Losses)])

In [ ]:
min([Loss_mean[i][2] for i in range(len(Loss_mean))])

In [ ]:
[Loss_mean[i][2] for i in range(len(Loss_mean))]